##             Jupyter Supported Interactive Data Processing Workflow 
AGU:
- https://agu2020fallmeeting-agu.ipostersessions.com/default.aspx?s=5A-96-79-A7-D6-DD-63-FF-D2-5A-B8-D7-5D-61-EB-D3

## Introduction
Raw hydrometeorological datasets contain errors, gaps and outliers that needs preprocessing. 
The objective of this work is developing an interactive preprocessing platform that enables acquiring and transforming publicly available raw hydrometeorological data to a ready to use  dataset. The interactive platform is at the core of the Comprehensive Hydrologic Observatory SEnsor Network CHOSEN dataset (Zhang et al. 2021 submitted to HP). 

In [1]:
import numpy as np
import pandas as pd
# from scipy import signal
import matplotlib.pyplot as plt
import ipywidgets
import datetime as dt
import copy
from pandas.plotting import register_matplotlib_converters
from sklearn import linear_model
from sklearn.metrics import r2_score
import copy
import os
import sys
from matplotlib import rcParams
from math import sqrt, pi
#import handcalcs.render

rcParams["font.size"]=14
plt.rcParams.update({'figure.max_open_warning': 0})
register_matplotlib_converters()
os.getcwd()


'G:\\My Drive\\DryCreek'

In [2]:
# local functions

sys.path.insert(1, './Functions')

from Source_QC_Widgets_functions_EM import regressorFunc, funcClimateCatalogWg,widgetInterpolation,widgetRegression,Date_to_float

## Input raw data table
The input data table needs to follow a standard naming. A column in the input table should have station name, variable name and measurement depths separated by underscore respectively.

In [3]:
# Watershed name and station names
watershed = 'DryCreek'
main_str = 'LG'

In [4]:
# Read the original data table
table = pd.read_csv('1_'+watershed+'_Download_Aggregation.csv',header = 0,index_col = 'DateTime',
                    parse_dates = True, infer_datetime_format = True,low_memory=False)
display(table.head(2))
display(table.tail(2))

,TL_Discharge,BSG_Discharge,LG_Discharge,C1E_Discharge,C1W_Discharge,C2E_Discharge,C2M_Discharge,BRW_Precipitation,LDP_Precipitation,SCR_Precipitation,...,LS_SoilTemperature_Pit1_30cm,LS_SoilTemperature_Pit2_2cm,LS_SoilTemperature_Pit2_15cm,LS_SoilTemperature_Pit2_30cm,LS_SoilTemperature_Pit3_2cm,LS_SoilTemperature_Pit3_15cm,LS_SoilTemperature_Pit3_30cm,LS_SoilTemperature_Pit4_2cm,LS_SoilTemperature_Pit4_15cm,LS_SoilTemperature_Pit4_30cm
DateTime,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,TL_Discharge,BSG_Discharge,LG_Discharge,C1E_Discharge,C1W_Discharge,C2E_Discharge,C2M_Discharge,BRW_Precipitation,LDP_Precipitation,SCR_Precipitation,...,LS_SoilTemperature_Pit1_30cm,LS_SoilTemperature_Pit2_2cm,LS_SoilTemperature_Pit2_15cm,LS_SoilTemperature_Pit2_30cm,LS_SoilTemperature_Pit3_2cm,LS_SoilTemperature_Pit3_15cm,LS_SoilTemperature_Pit3_30cm,LS_SoilTemperature_Pit4_2cm,LS_SoilTemperature_Pit4_15cm,LS_SoilTemperature_Pit4_30cm
DateTime,,,,,,,,,,,,,,,,,,,,,
2020-05-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.218182,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108333,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Double Check the station names
all_stations = table.columns.str.extract(r'([^_]+)')[0]
print('All stations names: ', all_stations.unique())
print ('  ')
nameStrflwStation=[]
nameHydrMetStation=[]
for i in np.arange(len(table.columns)):
    if table.columns[i][-9:]=='Discharge':  ### 
        if not all_stations[i] in nameStrflwStation:
            nameStrflwStation.append(all_stations[i]) ### 
    else:
        if not all_stations[i] in nameHydrMetStation:
            nameHydrMetStation.append(all_stations[i])  ### 

print('Discharge stations :',nameStrflwStation)
print('  ')
print('Meteorology stations:',nameHydrMetStation)                  

All stations names:  ['TL' 'BSG' 'LG' 'C1E' 'C1W' 'C2E' 'C2M' 'BRW' 'LDP' 'SCR' 'LW' 'HN' 'HS'
 'MHN' 'MHS' 'MLN' 'MLS' 'LN' 'LS']
  
Discharge stations : ['TL', 'BSG', 'LG', 'C1E', 'C1W', 'C2E', 'C2M']
  
Meteorology stations: ['BRW', 'LDP', 'SCR', 'TL', 'LW', 'HN', 'HS', 'MHN', 'MHS', 'MLN', 'MLS', 'LN', 'LS']


##  1. Trim the original table
This step adjusts the input table and removes lengthy missing values at the beigning and end of the input table.

In [6]:
t = table.notna() 
t = ~np.isnan(table)
col = len(t.columns)
b = np.zeros([table.shape[1]])
c = np.array([table.shape[0]] * table.shape[1])

for i in range(col):
    if any(t.iloc[:,i]): # Since some are empty
        b[i] = list(np.where(t.iloc[:,i] == True))[0][0] # the first non nan value location
        c[i] = list(np.where(t.iloc[:,i] == True))[0][-1] # the last non nan value location
        
st_tab = b.min()
table1 = table.iloc[int(b.min()):int(c.max()) + 1,:] 

# Display the trimmed table
display(table1.head(2))
display(table1.tail(2))
print('trimmed row number is ', int(table.shape[0] -  table1.shape[0]))

,TL_Discharge,BSG_Discharge,LG_Discharge,C1E_Discharge,C1W_Discharge,C2E_Discharge,C2M_Discharge,BRW_Precipitation,LDP_Precipitation,SCR_Precipitation,...,LS_SoilTemperature_Pit1_30cm,LS_SoilTemperature_Pit2_2cm,LS_SoilTemperature_Pit2_15cm,LS_SoilTemperature_Pit2_30cm,LS_SoilTemperature_Pit3_2cm,LS_SoilTemperature_Pit3_15cm,LS_SoilTemperature_Pit3_30cm,LS_SoilTemperature_Pit4_2cm,LS_SoilTemperature_Pit4_15cm,LS_SoilTemperature_Pit4_30cm
DateTime,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,TL_Discharge,BSG_Discharge,LG_Discharge,C1E_Discharge,C1W_Discharge,C2E_Discharge,C2M_Discharge,BRW_Precipitation,LDP_Precipitation,SCR_Precipitation,...,LS_SoilTemperature_Pit1_30cm,LS_SoilTemperature_Pit2_2cm,LS_SoilTemperature_Pit2_15cm,LS_SoilTemperature_Pit2_30cm,LS_SoilTemperature_Pit3_2cm,LS_SoilTemperature_Pit3_15cm,LS_SoilTemperature_Pit3_30cm,LS_SoilTemperature_Pit4_2cm,LS_SoilTemperature_Pit4_15cm,LS_SoilTemperature_Pit4_30cm
DateTime,,,,,,,,,,,,,,,,,,,,,
2020-05-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.218182,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108333,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


trimmed row number is  0


## Quantity of interest
QOI defines the variable name to be quality controlled and gap filled.

In [7]:
QOI = 'LS_SoilTemperature_Pit2_2cm'

# 2. Filling missing values

We gap fill the missing values using three alternatives methods:
1. Interpolation
2. Regression and 
3. Climate Catalog methods. 

### 2.1 Interpolation

- is the first gap filling technique adopted to fill short length missing values. It is not recommended for longer period missing values as it becomes unrealiable. This unrealiabilty can easily be detected by the interactive plots under the control parameter interpolation length limit (interplimit).

In [8]:
def plot_widgetInterpolation(QOI=QOI, interplimit=7, Intpmethod='time', Intplimit_direction='both',
                        xmin=None, xmax=None, ymin=None, ymax=None):
    
    y, yIntp = widgetInterpolation(table1,QOI, interplimit, 'time', 'both')
    
    indx = Date_to_float(y.index)
    
    fig, ax = plt.subplots(1, 1, figsize=(9, 4))
    ax.plot(indx, yIntp,'r',linewidth=2,label='Interpolated')
    ax.plot(indx, y, 'b',linewidth=2,label='Original')
    
    ax.set_xlabel("Year")
    ax.set_ylabel(QOI.split('_')[1])
    ax.set_title(QOI)
    ax.set_xlim([xmin,xmax])
    ax.set_ylim([ymin, ymax])
    ax.grid(color='gray', linestyle='-.', linewidth=1)
    #ax.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize='small')
    ax.legend(fontsize='small')
    
    # interaction responsiveness
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw_idle()

In [9]:
%matplotlib inline

# %matplotlib qt, - a separate plot
# %matplotlib notebook - the plot reside within the notebook 
# %matplotlib widget - for jupyter lab
Interpolation_widget = ipywidgets.interactive(
    plot_widgetInterpolation,
    
    QOI=table1,
    Intpmethod = ['linear','time','space','index','pad','nearest', 'zero', 'slinear', 'quadratic', 'cubic'],
    Intplimit_direction=['forward', 'backward', 'both'],
    
    interplimit=ipywidgets.IntSlider(min=2, max=100, value=7),
    xmin=ipywidgets.FloatText(value=2008),
    xmax=ipywidgets.FloatText(value=2018),
    ymin=ipywidgets.FloatText(value=0),
    ymax=ipywidgets.FloatText(value=50)
)
Interpolation_widget

interactive(children=(Dropdown(description='QOI', index=346, options=('TL_Discharge', 'BSG_Discharge', 'LG_Dis…

### 2.2 Regression


- longer period missing values are filled  by linear one at a time regression. Regression requires measurements from more than one station. By comparing these stations using correlation coefficient, a station with the highest correlation coefficient is adopted as a donor regression station. However, if the donor station's correlation coefficient is below a user defined regression thresold (Regthresh), regression will not be adopted. This threshold is identified interactively.



In [10]:
def plot_widgetRegression(QOI=QOI, RegThreshold=0.7, xmin=None, xmax=None, ymin=None, ymax=None):
    
    y, yReg = widgetRegression(table1, QOI, RegThreshold=RegThreshold)
    
    indx = Date_to_float(y.index)
    
    fig, ax = plt.subplots(1, 1, figsize=(9, 4))
    ax.plot(indx, yReg,'r',linewidth=2,label='After Regression')
    ax.plot(indx, y, 'b',linewidth=2,label='Raw Data')
    
    ax.set_xlabel("Year")
    ax.set_ylabel(QOI.split('_')[1])
    ax.set_title(QOI)
    ax.set_xlim([xmin,xmax])
    ax.set_ylim([ymin, ymax])
    ax.grid(color='gray', linestyle='-.', linewidth=1)
    #ax.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize='small')
    ax.legend(fontsize='small')
    
    # interaction responsiveness
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw_idle()
    # fig.gcf().canvas.set_window_title('Regression')

In [11]:
%matplotlib inline
# %matplotlib qt, - a separate plot
# %matplotlib notebook - the plot reside within the notebook with save and zooming options
# %matplotlib widget - for jupyter lab
Regression_widget = ipywidgets.interactive(
    plot_widgetRegression,
    
    QOI=table1,
    
    RegThreshold=ipywidgets.FloatSlider(min=.5, max=1, value=.9),
    xmin=ipywidgets.FloatText(value=2008),
    xmax=ipywidgets.FloatText(value=2018),
    ymin=ipywidgets.FloatText(value=0),
    ymax=ipywidgets.FloatText(value=50)
)
Regression_widget

interactive(children=(Dropdown(description='QOI', index=346, options=('TL_Discharge', 'BSG_Discharge', 'LG_Dis…

### 2.3 Climate catalog



- performs gap filling based on comparing the similarity of years. For instance, a missing value in a calendar day for a given year can be filled by a data from a year that has the highest correlation with the missing year among the other years plus a sample from a normal distribution with a standard deviation of all the years for the missing calandar day . Below is the mathematical formulation. 

- In using Climate Catalog, we set two interactive parameters: i) a minimum correaltion coefficient a  candidate year needs to satisfy to qualify as a donor year (corrThr) and ii) the minimum number of days the missing year has to have in order to perform Climate Catalog based gap filling (thrLen). 

In [12]:
def plot_widgetClimateCatalog(QOI=QOI, thrLen=200, corrThr=0.7, xmin=None, xmax=None, ymin=None, ymax=None):
    
    y_CC, yraw, indx_cc = funcClimateCatalogWg(table1, QOI, thrLen, corrThr)
    
    indx = Date_to_float(yraw.index)
    
    fig, ax = plt.subplots(1, 1, figsize=(9, 4))
    ax.plot(indx, y_CC,'r',linewidth=2,label='After Climate Catalog')
    ax.plot(indx, yraw, 'b',linewidth=2,label='Raw Data')
    
    ax.set_xlabel("Year")
    ax.set_ylabel(QOI.split('_')[1])
    ax.set_title(QOI)
    ax.set_xlim([xmin,xmax])
    ax.set_ylim([ymin, ymax])
    ax.grid(color='gray', linestyle='-.', linewidth=1)
    #ax.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize='small')
    ax.legend(fontsize='small')
    
    # interaction responsiveness
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw_idle()

In [13]:
%matplotlib inline

# %matplotlib qt, - a separate plot
# %matplotlib notebook - the plot reside within the notebook 
# %matplotlib widget - for jupyter lab
ClimateCatalog_widget = ipywidgets.interactive(
    plot_widgetClimateCatalog,
    
    QOI=table1,
    
    corrThr=ipywidgets.FloatSlider(min=.5, max=1., value=.7),
    thrLen=ipywidgets.IntSlider(min=1, max=365, value=7),
    xmin=ipywidgets.FloatText(value=2008),
    xmax=ipywidgets.FloatText(value=2018),
    ymin=ipywidgets.FloatText(value=0),
    ymax=ipywidgets.FloatText(value=50)
)
ClimateCatalog_widget

interactive(children=(Dropdown(description='QOI', index=346, options=('TL_Discharge', 'BSG_Discharge', 'LG_Dis…